# Tool definitions

In [20]:
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow, AgentInput, AgentOutput, \
    ToolCall, ToolCallResult, ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.llms.ollama import Ollama


In [21]:
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

def who_am_i() -> str:
    """ip_fetcher - Fetches current IP information"""
    import requests
    return requests.get("https://api.ipify.org?format=json").json()["ip"]

# Define LLMs

In [22]:
react_llm = AzureOpenAI(
                model='gpt-35-turbo',
                deployment_name='gpt-35-turbo',
                api_key='*********',
                azure_endpoint='https://dge-openai-2.openai.azure.com',
                api_version='2023-03-15-preview',
                supports_content_blocks=True,
           )

llm = AzureOpenAI(
                model='gpt-4o',
                deployment_name='gpt-4o',
                api_key='*********',
                azure_endpoint='https://dge-openai-2.openai.azure.com',
                api_version='2023-03-15-preview',
                supports_content_blocks=True,
           )

# Define Agents

In [32]:
calculator_agent = FunctionAgent (
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant.",
    tools=[
        FunctionTool.from_defaults(fn=add),
        FunctionTool.from_defaults(fn=subtract),
    ],
    llm=llm,
)

re_act_calculator_agent = FunctionAgent (
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="""
        You are an intelligent calculator. You can perform addition and subtraction by calling the appropriate functions.

        ## Instructions:
        1. Think step-by-step before taking action.
        2. Use the available functions (add, subtract) to perform calculations.
        3. If multiple calculations are needed, complete them before responding.
        4. Respond with the final answer only after all calculations are complete.

        ## Output Format:
        - **Thought:** Think about what needs to be done.
        - **Action:** Choose the correct function.
        - **Action Input:** Provide a valid JSON input to the function.
        - **Observation:** Receive function output.
        - **Final Answer:** Provide the computed result.
    """,
    tools=[
        FunctionTool.from_defaults(fn=add),
        FunctionTool.from_defaults(fn=subtract),
    ],
    llm=react_llm,
)

ip_fetcher = FunctionAgent (
    name="ip_fetcher",
    description="Retrieves public IP address of the user",
    system_prompt="You are a network expert that can fetch ip addresses. Do not respond before you retrieved the actual ip address.",
    tools=[
        FunctionTool.from_defaults(fn=who_am_i),
    ],
    llm=llm,
)

orchestrator_agent = FunctionAgent (
    name="orchestrator",
    description="Knows specialized agents and can hand over requests to them",
    system_prompt="You are an orchestrator. Given any user request you are able to understand his intention and delegate tasks to known agents. If you think the initial request is finally answered, you respond accordingly with a final response to the initial request.",
    llm=llm
)

# Define workflow

In [37]:
from llama_index.core.agent.workflow import AgentStream

workflow = AgentWorkflow (
    # agents=[orchestrator_agent, calculator_agent, ip_fetcher], root_agent="orchestrator"
    # agents=[orchestrator_agent, re_act_calculator_agent, ip_fetcher], root_agent="orchestrator"
    agents=[calculator_agent], root_agent="calculator"
)
response = workflow.run(user_msg="What is 3 + 5?")



final_answer = "N/A"
# response = workflow.run(user_msg="What is my ip address?")
async for event in response.stream_events():
    if isinstance(event, AgentInput):
        print("\n\n")
        print(" -- 1 --")
        print("Agent Input Event:")
        print("Input:", event.input)
        print("Current Agent:", event.current_agent_name)
    elif isinstance(event, AgentStream):
        # Agent thinking/tool calling response stream
        print("\n\n")
        print(" -- 2 --")
        print(event.delta)
        print(event.current_agent_name)
    elif isinstance(event, AgentOutput):
        print("\n\n")
        print(" -- 3 --")
        final_answer = event.response
        print(event.response)
        print(event.tool_calls)
        print(event.raw)
        print(event.current_agent_name)
    elif isinstance(event, ToolCall):
        print("\n\n")
        print(" -- 4 --")
        # Tool being called
        print(event.tool_name)
        print(event.tool_kwargs)
    elif isinstance(event, ToolCallResult):
        print("\n\n")
        print(" -- 5 --")
        # Result of tool call
        print(event.tool_output)
    else:
        print("\n\n")
        print(" -- 6 --")
        print (event)

print ("\n#####################\n")
print(final_answer)





 -- 1 --
Agent Input Event:
Input: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are a calculator assistant.')]), ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='What is 3 + 5?')])]
Current Agent: calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 2 --

calculator



 -- 3 --
assistant: None
[ToolSelection(tool_id='call_H7qlsm0C8orlkLANWUpzd6ZR', tool_name='add', tool_kwargs={'a': 3, 'b': 5})]
{'id': 'chatcmpl-Azhn4mxg0TgPbIFS5qmUIzTStdLF3', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None}], 'created': 1739269702, 'model':